## IMPORT STATEMENTS

In [206]:
import pandas as pd
import numpy as np
import nltk
import re 
import os
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import LancasterStemmer
from collections import defaultdict
import pickle
# import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
import tensorflow_hub as hub
import numpy as np
from scipy.spatial.distance import cosine
from itertools import chain
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## STOP WORDS INITIALIZATION

In [207]:
stop_words = set()
languages = ['english','hindi']
for lang in languages:
    with open(lang+".txt",'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.rstrip()
            stop_words.add(line)

print(stop_words)

{'become', 'four', 'hers', 'abroad', 'वे', 'act', 'haven', 'somehow', 'way', 'apparently', 'pm', 'nos', 'doing', 'readily', 'kz', 'ao', 'approximately', 'ml', 'पे', 'into', 'tn', 'eg', "we'll", 'faces', 'opening', 'thatve', 'thered', 'hardly', 'इन्हें', 'backward', 'front', 'members', "can't", 'pmid', 'घर', 'msie', 'opened', 'her', 'गया', 'जब', 'संग', 'corresponding', 'wants', 'text', 'says', 'tg', 'welcome', 'across', 'long', 'various', 'hk', 'rooms', 'इसके', 'successfully', 'strongly', 'they', 'किसे', 'thinks', 'somewhere', 'सकते', 'es', 'unlike', 'za', 'thereve', 'aint', 'lately', 'cd', 'sm', 'fewer', 'td', 'hereupon', 'computer', 'downed', 'state', 'necessarily', "c's", 'mv', 'where', 'need', 'uz', 'hundred', 'yours', 'described', 'two', "hasn't", 'meanwhile', 'farther', 'h', 'k', 'myse”', "who's", 'w', 'pp', 'कुल', 'mrs', 'anything', 'small', 'there', 'please', 'half', 'is', 'selves', 'evermore', 'mq', 'जिन', 'pages', 'proud', 'होती', 'इसकी', 'far', 'उसके', 'yourself', 'o', 'si', 

## BASEWORDS INITILIAZATION

In [208]:
base_tweet = list()
base_tweet.append("No more funding to terrorists, naxalites and underworld. great step towards eliminating all such practices.")
print(base_tweet[0])

No more funding to terrorists, naxalites and underworld. great step towards eliminating all such practices.


## DATA PREPROCESSING

In [225]:
def data_cleaning(tweet_array,tweet_id,length):
  # using nltk twitter tokenizer
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lancaster=LancasterStemmer()
    processed_review=[]
    mapping = dict()
    single_review = ""
    for index in range(0,length):
        single_review = tweet_array[index]
        # remove html tags
        single_review = re.sub('<.*?>',' ',single_review)
        # remove single characters
        single_review = re.sub('\s+[a-zA-Z]\s+',' ', single_review)
        # remove multiple spaces 
        single_review = re.sub('\s+',' ', single_review)
        # use twitter tokenizer
        tk = TweetTokenizer()
        word_tokens = tk.tokenize(single_review)
        lemmatized_sentence = " ".join(lemmatizer.lemmatize(token) for token in word_tokens if token not in stop_words)
        processed_review.append(lemmatized_sentence)
        mapping[tweet_id[index]]=single_review

    
    return processed_review, mapping

In [226]:
# This function takes tweets as input and preprocess it consider tweet id
def data_preprocessing(df):
  # finding different languages and considering only english and hindi
  print("unique languages are",df.language.unique())
  en_df = df.loc[df['language']=='en']
  hi_df = df.loc[df['language']=='hi']
  print("english tweets shape is",en_df.shape)
  print("hindi tweets shape is",hi_df.shape)
  english_content = np.array(en_df.content)
  hindi_content = np.array(hi_df.content)
  english_tweet_id = np.array(en_df.tweet_id)
  hindi_tweet_id = np.array(hi_df.tweet_id)
  length1 = en_df.shape[0]
  length2 = hi_df.shape[0]
  english_tweets, en_id_content = data_cleaning(english_content,english_tweet_id, length1)
  hindi_tweets, hi_id_content  = data_cleaning(hindi_content,hindi_tweet_id,length2)
  return english_tweets, hindi_tweets, en_id_content, hi_id_content


In [227]:
india_propaganda = 'https://raw.githubusercontent.com/abhiram144/Copy-Pasta-Analysis/main/hash_tag_csv/IndiaDefeatsBlackMoney.csv'
df = pd.read_csv(india_propaganda)
english_tweets, hindi_tweets, en_mapping, hi_mapping = data_preprocessing(df)

unique languages are ['en' 'hi' 'und' 'fr' 'et' 'cy' 'es' 'tl' 'in' 'mr' 'gu' 'pt' 'sv' 'vi'
 'ml' 'ta' 'nl' 'ne' 'ro' 'cs' 'is' 'it' 'ar' 'ht' 'fi' 'da' 'kn' 'sl'
 'tr' 'pl' 'ca' 'de' 'ko' 'hu' 'no' 'eu']
english tweets shape is (9063, 9)
hindi tweets shape is (1665, 9)


In [212]:
print(len(en_mapping))

9063


In [187]:
preprocessed_tweets = list()
t, id = data_cleaning(base_tweet,[1],len(base_tweet))
print(t)

['No funding terrorist , naxalites underworld . step eliminating practice .']


In [213]:
print(id)

{1: 'No more funding to terrorists, naxalites and underworld. great step towards eliminating all such practices.'}


## USER DATA LOADING

In [157]:
party_url = 'https://raw.githubusercontent.com/abhiram144/Copy-Pasta-Analysis/main/user_tweets/user_data.csv'
df_party_names = pd.read_csv(party_url)
user_details_url = 'https://raw.githubusercontent.com/abhiram144/Copy-Pasta-Analysis/main/user_tweets/user_details.csv'
df_user_data = pd.read_csv(user_details_url)

In [158]:
df_party_names.head()

,Unnamed: 0,verified,location,id,url,public_metrics,protected,party,username,name,created_at,description
0,0,True,"Punjab, India",3153134184,NaN,"{'followers_count': 58375, 'following_count': ...",False,BJP,sunilkjakhar,Sunil Jakhar,2015-04-13T16:14:06.000Z,NaN
1,1,True,NEW DELHI,45795876,NaN,"{'followers_count': 21332, 'following_count': ...",False,BJP,rpsinghkhalsa,RP Singh National Spokesperson BJP,2009-06-09T08:33:22.000Z,"Advertising Professional.\nPatron, Gali Pathsh..."
2,2,True,Dehradun,1297926962,https://t.co/SkirgMiB7f,"{'followers_count': 1907, 'following_count': 2...",False,BJP,bjpnareshbansal,Naresh Bansal,2013-03-25T06:31:52.000Z,Member of Parliament Rajya Sabha | Ex. General...
3,3,True,pokhari chamoli uttarakhand,2692838006,https://t.co/9j75o0vuWo,"{'followers_count': 5481, 'following_count': 3...",False,BJP,mahendrabhatbjp,Mahendra Bhatt MLA,2014-07-30T13:02:25.000Z,महेंद्र भट्ट विधायक बद्रीनाथ विधानसभा उत्तराखं...
4,4,True,"Yamkeshwar, India",1028186624218288128,https://t.co/SxS5stdGzU,"{'followers_count': 2602, 'following_count': 1...",False,BJP,RituKhanduriBJP,Ritu Khanduri Bhushan,2018-08-11T07:49:28.000Z,Official Account of Smt. Ritu Khanduri Bhushan...


In [159]:
df_user_data.head()

,username,id,displayname,followersCount,friendsCount,location
0,BJP4JnK,1864685238,BJP Jammu & Kashmir,86433,222,Jammu & Kashmir
1,GulabMatiala,2922933098,Gulab Singh,38727,716,"New Delhi, India"
2,guptaravinder71,2204739805,Ravinder Gupta 🇮🇳,23358,1234,"Karol Bagh, New Delhi"
3,MaheishGirri,425973946,Maheish Girri,133913,657,"New Delhi, India"
4,shweta_shalini,2366619296,Shweta Shalini,22684,374,Mumbai


## PERFORM EMBEDDING

In [214]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
# model = hub.load(module_url)

In [215]:
# !pip install sentence-transformers

In [216]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [217]:
tweet_embeddings = sbert_model.encode(english_tweets)
print(tweet_embeddings.shape)

(9063, 768)


In [218]:
base_embeddings = sbert_model.encode(t)
print(base_embeddings.shape)

(1, 768)


## CREATE MAPPING BETWEEN TWEET_ID AND EMBEDDING

In [230]:
# This function creates mapping between embedding and tweet id
from collections import defaultdict
def reverse_mapping(tweet_embeddings,en_mapping):
  rev_map = defaultdict(list)
  index = 0
  for tweet_id in en_mapping.keys():
    tupple = tuple(tweet_embeddings[index])
    rev_map[tupple].append(tweet_id)
    index = index+1 
  print(index)
  return rev_map

In [231]:
rev_map = reverse_mapping(tweet_embeddings,en_mapping)
print(len(rev_map))

9063
6955


## Cosine similarity

In [244]:
from sklearn.metrics.pairwise import cosine_similarity
def similarity(base_embeddings,tweet_embeddings):
  temp_np_array = np.zeros((len(tweet_embeddings),768))
  i = 0
  for k in tweet_embeddings:
    k = np.array(k)
    temp_np_array[i]=k
    i=i+1
  c_sim=cosine_similarity(base_embeddings,temp_np_array)
  return c_sim

In [245]:
sim=similarity(base_embeddings,rev_map.keys())
print(sim.shape)

(1, 6955)


In [246]:
sim_list = sim.tolist()

In [250]:
f_list = sim_list[0]
print(len(f_list))
result = list()
# print(len(tweet_embeddings))
k_list = list(rev_map.keys())
for index in range(len(f_list)):
  if f_list[index] >= 0.80:
      result.append(k_list[index])


print(len(result))

6955
11


In [256]:
c_p_tweet = list()
for r in result:
  tweet_id = rev_map[tuple(r)]
  for t in tweet_id:
    c_p_tweet.append(t)
print(len(c_p_tweet))

44


In [241]:
print("total number of copy pasted",len(c_p_tweet))

total number of copy pasted 29


In [ ]:
df.head()

In [ ]:
new_df = df.loc[df['tweet_id'].isin(tweet_id)]

In [ ]:
new_df[['date', 'time']] = new_df['date'].str.split('T', 1, expand=True)


In [ ]:
print(len(new_df))

In [ ]:
new_df.head()

In [ ]:
t_df = new_df.sort_values(by='date')
t_df.head()